In [6]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
import time
import matplotlib.pyplot as plt



## print in readable format
def Print(object):
    for i in range(object.__len__()):
        print(object[i], end =' ')
        if(i%10==0): print('\n')




## find index of string columns
def FindStrIndex(df):
    StrIndex = []
    for i in range(len(list(df))):
        if (isinstance(df.iloc[1, i], str)):
            StrIndex.append(i)
    return StrIndex



## Encoding categorical data
def MakeEncoded(obj):
    le = preprocessing.LabelEncoder()
    for column_name in obj.columns:
        if obj[column_name].dtype == object:
            obj[column_name] = le.fit_transform(obj[column_name])
        else:
            pass
    return obj



In [10]:
Data = pd.read_csv('..//Data//MergedDesignMatLabel_SecondStruct_LenFilter.csv')
# Print(list(Data))
Data = Data.drop(['Unnamed: 0','ic','ev' ,'X', 'id', 'annotation','rnaType','DB'],axis=1)
print(Data.head())




    chr                                                seq  length   a   c  \
0  chr1                    GGTGAGGCCGAGAAGGCAACCGCGAAAAGGA      31  35  19   
1  chr1                                   TAAGAGGAGTTGTTTG      16  25   0   
2  chr1  CGCCAGAGGAGCAGCAGCGCTTGTGCAAACCGGGAAGATGGCGGCC...     110  16  30   
3  chr1                               GACCAAGCTGGAGAATGATT      20  35  15   
4  chr1                               GACGGAGCGAGGAAAAAAAG      20  50  10   

    g   t strand  AAAA  AAAC  ...   C110  C111  G000  G001  G010  G011  G100  \
0  41   3      +     1     0  ...      3     2     4     1     0     2     3   
1  37  37      +     0     0  ...      0     0     5     0     0     0     0   
2  40  13      +     0     1  ...      5    15     9     4     0     7     4   
3  30  20      +     0     0  ...      0     1     3     0     0     0     0   
4  40   0      +     4     0  ...      0     0     6     0     0     0     0   

   G101  G110  G111  
0     0     0     2  
1     

In [11]:
### split data to design matrix and labels
X = Data.drop(['label'],axis=1)
y = Data['label']


### split test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)


le = preprocessing.LabelEncoder()
X_train = MakeEncoded(X_train.loc[:,:])
y_train = le.fit_transform(y_train)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
X_train.head()

,chr,seq,length,a,c,g,t,strand,AAAA,AAAC,...,C110,C111,G000,G001,G010,G011,G100,G101,G110,G111
26939,23,7158,14,14,28,28,28,0,0,0,...,0,1,1,0,0,1,0,0,0,1
22079,18,13681,23,21,26,30,21,1,0,0,...,0,2,0,0,0,0,2,0,1,3
11350,9,10300,19,26,10,47,15,1,0,0,...,0,1,4,0,0,0,1,0,1,1
23440,19,5288,18,38,27,27,5,1,2,0,...,1,0,3,0,0,1,0,0,1,0
3975,2,15746,15,20,46,20,13,0,0,0,...,0,0,0,0,0,1,1,0,1,0


In [ ]:




## SVM linear kernel- unbalanced
tic = time.time()
SvmLinearUnBal = SVC(kernel='linear')
SvmLinearUnBal.fit(X_train, y_train)
toc = time.time() - tic
y_pred = SvmLinearUnBal.predict(MakeEncoded(X_test.loc[:,]))

### evaluation
metrics.confusion_matrix(le.fit_transform(y_test),y_pred)
metrics.classification_report(le.fit_transform(y_test),y_pred)
metrics.accuracy_score(le.fit_transform(y_test),y_pred)

# toDo: check average micro/macro
# toDo: recall for +/- classes, sens/spec?
# toDo: how to visualize the evaluation results?
metrics.recall_score(le.fit_transform(y_test), y_pred, average='macro')
#################



# SVM linear kernel- balanced
tic = time.time()
SvmLinearBal = SVC(kernel='linear', class_weight='balanced')
SvmLinearBal.fit(X_train, y_train)
toc = time.time() - tic



# SVM Radial kernel- balanced
tic = time.time()
SvmRadBal = SVC(kernel='rbf', class_weight='balanced')
SvmRadBal.fit(X_train, y_train)
toc = time.time() - tic


################ Hyperparameter tuning